In [1]:
%matplotlib inline
import os
import numpy as np
import xarray as xr
from datetime import datetime

from itertools import product

import matplotlib.pyplot as plt

import pop_tools
import util

Cannot write to data cache '/glade/p/cesmdata/cseg'. Will not be able to download remote data files. Use environment variable 'CESMDATAROOT' to specify another directory.


## Generate POP grid files

In [2]:
for grid_name in util.POP_grids:
    scrip_grid_file = util.file_name_grid(grid_name)

    if not os.path.exists(scrip_grid_file):            
        print(f'generating {scrip_grid_file}')
        
        grid = pop_tools.get_grid(grid_name)
        del grid.attrs['region_mask_regions']
        grid.to_netcdf(scrip_grid_file)



Generate remapping files

In [3]:
#%%bash
# qsub not available from Casper
#qsub < esmf_gen_weights_etopo1_to_POP.pbs

In [4]:
#%%bash
# qsub not available from Casper
#qsub < esmf_gen_weights_POP_to_POP.pbs